In [2]:
import json
import pandas as pd
import numpy as np
import random
import os

# --- KONFIGURASI ---
GEOJSON_PATH = "data/kecamatan.geojson"
OUTPUT_DIR = "data"
START_DATE = "2024-01-01"
END_DATE = "2024-03-31"

def get_kecamatan_from_geojson():
    """Membaca daftar kecamatan dari GeoJSON (Source of Truth)."""
    if not os.path.exists(GEOJSON_PATH):
        print(f"❌ Error: File {GEOJSON_PATH} tidak ditemukan.")
        return []
    
    print(f"📂 Membaca GeoJSON dari {GEOJSON_PATH}...")
    with open(GEOJSON_PATH, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    kecamatan_list = set()
    for feature in data.get('features', []):
        props = feature.get('properties', {})
        name = props.get('district') or props.get('kecamatan') or props.get('name')
        if name:
            clean_name = name.strip().upper().replace('\xa0', ' ')
            kecamatan_list.add(clean_name)
            
    return sorted(list(kecamatan_list))

def generate_sipsn_random(kecamatan_list):
    """Membuat data SIPSN dengan variasi demografi yang ekstrem."""
    print("🛠️  Generating sipsn.csv (High Variance)...")
    data = []
    for kec in kecamatan_list:
        # Acak kategori wilayah: Kecil, Sedang, Besar
        category = random.choice(['small', 'medium', 'large', 'mega'])
        
        if category == 'small':
            luas = random.uniform(3.0, 8.0)
            penduduk = int(luas * random.uniform(5000, 10000))
        elif category == 'medium':
            luas = random.uniform(8.0, 15.0)
            penduduk = int(luas * random.uniform(10000, 20000))
        elif category == 'large':
            luas = random.uniform(15.0, 25.0)
            penduduk = int(luas * random.uniform(15000, 30000))
        else: # Mega (Sangat Padat)
            luas = random.uniform(10.0, 20.0)
            penduduk = int(luas * random.uniform(30000, 50000))
            
        armada_total = int(penduduk / 4000)
        if armada_total < 2: armada_total = 2
        
        data.append({
            "kecamatan": kec,
            "armada_total": armada_total,
            "armada_operasional": int(armada_total * random.uniform(0.7, 0.95)), # Variasi availability
            "ritase_harian": round(random.uniform(1.0, 4.0), 1), # Variasi ritase tinggi
            "kapasitas_m3": round(armada_total * random.uniform(5, 12), 1),
            "penduduk": penduduk,
            "luas_km2": round(luas, 2)
        })
    pd.DataFrame(data).to_csv(f"{OUTPUT_DIR}/sipsn.csv", index=False)
    print("✅ sipsn.csv generated.")

def generate_waste_random(kecamatan_list):
    """
    Membuat data sampah yang SANGAT ACAK (High Randomness).
    """
    print("🛠️  Generating waste.csv (High Randomness)...")
    
    date_range = pd.date_range(start=START_DATE, end=END_DATE)
    
    # 1. Tetapkan Basis Volume yang Sangat Bervariasi (20 ton s/d 200 ton)
    # Ini menjamin peta akan warna-warni (ada yang merah tua, ada yang pink muda)
    kec_base_volume = {kec: random.uniform(20, 200) for kec in kecamatan_list}
    
    data = []
    
    # Opsi metadata acak
    sumber_opts = ["Rumah Tangga", "Pasar", "Komersial", "Industri Kecil", "Taman Kota"]
    jenis_opts = ["Organik", "Anorganik", "B3", "Residu"]

    for date in date_range:
        # 2. Faktor Hari Ekstrem
        weekday = date.dayofweek
        if weekday == 0: # Senin: Lonjakan parah
            day_factor = random.uniform(1.3, 1.5)
        elif weekday == 6: # Minggu: Drop parah
            day_factor = random.uniform(0.6, 0.8)
        else:
            day_factor = random.uniform(0.9, 1.1)

        for kec in kecamatan_list:
            base = kec_base_volume[kec]
            
            # 3. Random Noise Tinggi (± 30%)
            noise = random.uniform(0.7, 1.3)
            
            # 4. EVENT KEJUTAN (5% kemungkinan volume naik 2x lipat)
            # Simulasi banjir, event besar, atau tumpukan sampah liar
            spike = 1.0
            if random.random() < 0.05: 
                spike = random.uniform(1.5, 2.0)
            
            final_volume = base * day_factor * noise * spike
            
            # Pastikan tidak negatif
            if final_volume < 5: final_volume = 5
            
            data.append({
                "tanggal": date.strftime("%Y-%m-%d"),
                "kecamatan": kec,
                "volume_ton": round(final_volume, 2),
                "jenis_sampah": random.choice(jenis_opts), # Acak jenis sampah
                "sumber_sampah": random.choice(sumber_opts) # Acak sumber
            })
            
    pd.DataFrame(data).to_csv(f"{OUTPUT_DIR}/waste.csv", index=False)
    print("✅ waste.csv generated (Random).")

if __name__ == "__main__":
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)
        
    kec_list = get_kecamatan_from_geojson()
    
    if kec_list:
        generate_sipsn_random(kec_list)
        generate_waste_random(kec_list)
        print("\n🎉 Data Dummy RANDOM Berhasil Dibuat!")
        print("Peta Anda sekarang akan terlihat sangat bervariasi dan grafik tren lebih dinamis.")
    else:
        print("❌ Gagal. GeoJSON tidak ditemukan.")

📂 Membaca GeoJSON dari data/kecamatan.geojson...
🛠️  Generating sipsn.csv (High Variance)...
✅ sipsn.csv generated.
🛠️  Generating waste.csv (High Randomness)...
✅ waste.csv generated (Random).

🎉 Data Dummy RANDOM Berhasil Dibuat!
Peta Anda sekarang akan terlihat sangat bervariasi dan grafik tren lebih dinamis.
